In [1]:
import os
os.chdir('c:\\Users\\Vigneswaran\\OneDrive - Datanetiix Solutions Inc\\Desktop\\Diesease_Prognosis')

In [2]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

# Retrieve variables
os.environ["MLFLOW_TRACKING_URI"] = os.getenv("MLFLOW_TRACKING_URI")
os.environ["MLFLOW_TRACKING_USERNAME"] = os.getenv("MLFLOW_TRACKING_USERNAME")
os.environ["MLFLOW_TRACKING_PASSWORD"] = os.getenv("MLFLOW_TRACKING_PASSWORD")

# Now you can use the variables as needed
print(os.environ["MLFLOW_TRACKING_URI"])

https://dagshub.com/vignesh2914/Disease_Prognosis_Prediction.mlflow


In [4]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import mlflow
import mlflow.sklearn
import dagshub
from pathlib import Path
from diseaseprognosis.utils.common import save_json

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path  ## typically we have (accuracy, precision, recall, F1 score, etc.)
    target_column: str
    mlflow_uri: str  # Add this line


In [6]:
from diseaseprognosis.constants import *
from diseaseprognosis.utils.common import read_yaml, create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/vignesh2914/Disease_Prognosis_Prediction.mlflow",     
        )

        return model_evaluation_config

In [8]:

import os
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        score = accuracy_score(actual, pred)
        return score
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            score = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"score": score}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("score", score)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-09 21:14:23,509: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-09 21:14:23,512: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-09 21:14:23,531: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-09 21:14:23,540: INFO: common: created directory at: artifacts]
[2024-12-09 21:14:23,540: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-09 21:14:25,329: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/12/09 21:14:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'RandomForestModel' already exists. Creating a new version of this model...
2024/12/09 21:14:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestModel, version 7
Created version '7' of model 'RandomForestModel'.
2024/12/09 21:14:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run adventurous-bass-836 at: https://dagshub.com/vignesh2914/Disease_Prognosis_Prediction.mlflow/#/experiments/0/runs/668094632a944790a47eaf2b8acb9591.
2024/12/09 21:14:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/vignesh2914/Disease_Prognosis_Prediction.mlflow/#/experiments/0.
